In [ ]:
# default_exp evaluate

In [ ]:
%load_ext autoreload
%autoreload 2

# Evaluate

> This module contains functions to infer on the test set and generate the final submission file for Kaggle.

In [ ]:
#hide
from nbdev.showdoc import *
from plant_pathology.utils import *
from plant_pathology.dataset import *
from wwf.vision.timm import *
import timm

In [ ]:
#export
from fastai.vision.all import *
from typing import *

## Infer on Test Set

In [ ]:
#export
def infer_on_test_set(learn: Learner, tta: bool=False, path_test: Path=Path("/home/brandon/projects/plant_pathology/data/test.csv")) -> Tensor:
    df_test = pd.read_csv(path_test)
    test_dl = learn.dls.test_dl(df_test)
    preds, _ = (learn.tta if tta else learn.get_preds)(dl=test_dl)
    return preds

In [ ]:
#hide
path, df = load_data()
dls = get_dls(path, df, presize=32, resize=32, bs=512)

In [ ]:
learn = cnn_learner(dls, resnet18)
preds = infer_on_test_set(learn, tta=True)

In [ ]:
test_close(preds.sum(1), 1.)  # cnn_learner applies softmax after tta

In [ ]:
timm_learn = timm_learner(dls, "seresnext26d_32x4d")
preds = infer_on_test_set(timm_learn, tta=True)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/seresnext26d_32x4d-80fa48a3.pth" to /home/brandon/.cache/torch/hub/checkpoints/seresnext26d_32x4d-80fa48a3.pth


In [ ]:
preds

tensor([[0.3558, 0.1955, 0.2521, 0.1966],
        [0.3295, 0.1942, 0.2538, 0.2225],
        [0.3740, 0.2734, 0.1763, 0.1763],
        ...,
        [0.2949, 0.2883, 0.2412, 0.1756],
        [0.3476, 0.2481, 0.2396, 0.1647],
        [0.3084, 0.1970, 0.3568, 0.1378]])

## Format Submission

In [ ]:
#export
def format_submission(preds: Tensor, save_path: Union[Path, str], path_data: Path=Path("/home/brandon/projects/plant_pathology/data")) -> Path:
    submission = pd.read_csv(path_data/"sample_submission.csv")
    
    # Update cols with preds
    submission["healthy"] = preds[:, 0]
    submission["multiple_diseases"] = preds[:, 1]
    submission["rust"] = preds[:, 2]
    submission["scab"] = preds[:, 3]
    
    # Make parent dirs
    save_path = Path(save_path)
    Path(save_path.parent).mkdir(parents=True, exist_ok=True)
    submission.to_csv(save_path, index=False)
    return save_path

In [ ]:
fake_preds = torch.zeros(1821, 4)
save_path = format_submission(fake_preds, "TESTING.csv"); save_path

Path('TESTING.csv')

In [ ]:
assert save_path.exists()

In [ ]:
save_path.unlink()

## All-in-One Learner -> Submission Function

In [ ]:
#export
def evaluate(learn: Learner, save_path: Union[Path, str]=Path("./submission.csv"), tta: bool=False) -> Path:
    """Takes trained learner, evaluates on test set, formats and saves submission.csv."""
    preds = infer_on_test_set(learn, tta=tta)
    return format_submission(preds, save_path)

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_utils.ipynb.
Converted 01_dataset.ipynb.
Converted 02_evaluate.ipynb.
Converted 03_train.ipynb.
Converted 04_generate_pseudo_labels.ipynb.
Converted 05_self_knowledge_distillation.ipynb.
Converted index.ipynb.
